## W266 Final Project: Project Milestone

### Classifying the Political Ideology of News Articles

#### Matt Acconciamessa and Megan Pera



In [ ]:
%matplotlib inline

# Import necessary libraries
import pickle
import numpy as np
from scipy import sparse
import collections
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import *
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

### Loading, Cleaning and Exploring Data

In [ ]:
# Load and save data into liberal, conservative and neutral objects
[lib, con, neutral] = pickle.load(open('ibcData.pkl', 'rb'))

In [ ]:
# Data samples, by classification
print ('Liberal examples (out of ', len(lib), ' sentences): ')
for tree in lib[0:5]:
    print(tree.get_words())
    
print ('\nConservative examples (out of ', len(con), ' sentences): ')
for tree in con[0:5]:
    print (tree.get_words())
    
print ('\nNeutral examples (out of ', len(neutral), ' sentences): ')
for tree in neutral[0:5]:
    print (tree.get_words())

In [ ]:
# Formatting data into workable arrays
liberal = np.array(lib)
conserv = np.array(con)
neut = np.array(neutral)

# Seprating data and labels
def separate_data_and_labels(label_class):
    labels = []
    data = []
    for i in range(len(label_class)):
        for node in label_class[i]:
            if hasattr(node, 'label'):
                data.append(node.get_words())
                labels.append(node.label)
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

lib_data, lib_labs = separate_data_and_labels(liberal)
con_data, con_labs = separate_data_and_labels(conserv)
neut_data, neut_labs = separate_data_and_labels(neut)

In [ ]:
print('Examples:')
print ('\n Liberal')
print(lib_data[0],'\n',lib_labs[0:10])
print ('\n Conservative')
print(con_data[0],'\n',con_labs[0:10])
print ('\n Neutral')
print(neut_data[0],'\n',neut_labs[0:10])

In [ ]:
# Combining into one dataset
data_all = np.concatenate((neut_data, lib_data, con_data), axis=0)
labs_all = np.concatenate((neut_labs, lib_labs, con_labs), axis=0)

print (data_all.shape)
print (labs_all.shape)

In [ ]:
# Randomly mixing data&labels so that they can be split into test and train
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

data_all, labs_all = shuffle_in_unison(data_all, labs_all)

# @Matt
# Should we be including a dev set as well, and reserve a test set for the very last iteration of each model?

In [ ]:
# Split data into test (20%) and train (80%)
slice = int(.8*labs_all.shape[0])
data_train = data_all[:slice]
labs_train = labs_all[:slice]
data_test = data_all[slice:]
labs_test = labs_all[slice:]
print(labs_all.shape)
print(labs_test.shape)
print(labs_train.shape)

In [ ]:
# Turning dataset into word tokens
count_vect = CountVectorizer()
data = count_vect.fit_transform(data_train).toarray()
vocab = count_vect.get_feature_names()

# Counting the number of times each word appears
np.clip(data,0,1, out = data) #make sure each word only appears once in the array
dist = np.sum(data, axis = 0) #sum the columns
counts = list(zip(vocab,dist)) #zip counts and words together

# Total vocab size and word count
print("Total word count:",np.sum(dist))
print("Vocabulary size:",len(vocab))

In [ ]:
# Printing out the 20 most popular words
counts = sorted(counts, key=lambda x: x[1], reverse=True) 
counts[0:20]

In [ ]:
# Plotting top 50 results
ordered = list(zip(*counts))
x = ordered[0][:50] #counts
y = ordered[1][:50] #words

# Plotting figure
fig = plt.figure(figsize=(15.0,6.0))
indexes = np.arange(50)
width = .25
plt.bar(indexes, y, width)
plt.xticks(indexes + width * 0.5, x,rotation=70)
plt.show()

### Baseline model: Multinomial Naive Bayes


In [ ]:
# This model predicts the political leanings of sentences and sub-sentences

# Training the model
vect = CountVectorizer()
train_vocab = vect.fit_transform(data_train)
test_vocab = vect.transform(data_test)

# Scoring the model
print("")
print("Multinomial Naive Bayes:")
for a in [0.0001, 0.01, .05, 0.1, 0.2, 1.0]:
    mnb = MultinomialNB(alpha=a)
    mnb.fit(train_vocab, labs_train)
    mnbpreds = mnb.predict(test_vocab)
    print("alpha:", a, "F1:", metrics.f1_score(labs_test,mnbpreds,average='weighted'))

In [ ]:
# Showing examples for alpha = 0.001
mnb = MultinomialNB(alpha=0.001)
mnb.fit(train_vocab, labs_train)
mnbpreds = mnb.predict(test_vocab)
mnbpred_prob = mnb.predict_proba(test_vocab)
probs = list(zip(data_test.tolist(),mnbpreds.tolist(),mnbpred_prob.tolist()))

for i in range(0,5):
    print('Sentence:',probs[i][0])
    print('Actual Label:',labs_test[i])
    print('Predicted Label:',probs[i][1])
    print('Predicted Label Probability:', max(probs[i][2]),'\n')

In [ ]:
# Finding and printing out mistakes
errors = []
for i in range(0,len(probs)):
    if labs_test[i] == probs[i][1]:
        pass
    else:
        errors.append(i)
        
print('MNB missclassified',len(errors),'sentences','\n')

for i in errors[0:5]:
    print('Sentence:',probs[i][0])
    print('Actual Label:',labs_test[i])
    print('Predicted Label:',probs[i][1])
    print('Predicted Label Probability:', max(probs[i][2]),'\n')

### Building a pipeline for News Articles to Score with Models

In [1]:
from newspaper3k import Article

ImportError: No module named 'newspaper3k'